# IMPORT FUNCFIONS

In [1]:
import pandas as pd
import numpy as np
import networkx as nx
import itertools
import pickle
import os
import random
from sklearn.model_selection import train_test_split

from sklearn.metrics import average_precision_score
from sklearn import metrics

from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

from ge import LINE

# READ DATA

In [2]:
def get_edges_dict(traindata):
    if os.path.isfile("edgesdic.pkl"):
        f = open("edgesdic.pkl","rb")
        edges = pickle.load(f)
        f.close()
        return edges
    else:
        edges = dict()
        for edge in range(len(traindata)): 
            edges[(traindata.loc[[edge]]['id_1'].values[0],traindata.loc[[edge]]['id_2'].values[0])] = 1 
            edges[(traindata.loc[[edge]]['id_2'].values[0],traindata.loc[[edge]]['id_1'].values[0])] = 1  
        f = open("edgesdic.pkl","wb")
        pickle.dump(edges,f)
        f.close()
        return edges

def get_negative_edges(traindata,g):
    edges = get_edges_dict(traindata)
    # for missing edges.
    negative_edges = set([])
    maxNodenum = max(max(traindata['id_1']),max(traindata['id_2']))
    #產生與原圖同edge數目的negative edges
    while (len(negative_edges)<len(traindata)):
        node1=random.randint(0, maxNodenum) 
        node2=random.randint(0, maxNodenum) 
        tmp = edges.get((node1,node2),0) #edge不在graph裡標為0
        if tmp == 0 and node1!=node2 and g.has_node(node1) and g.has_node(node2): # if edge不在graph裡才要做處理
            negative_edges.add((node1,node2))
        else:
            continue
    return negative_edges

# MODELS

In [3]:
def combine_embedding(data,embeddings):
    i=0
    X = []
    for node1,node2 in data:
        X.append(np.concatenate((data[i],embeddings[int(node1)],embeddings[int(node2)])))
   # print(embeddings[str(int(data[0]))])
        i+=1
    return X

In [4]:
all_ap = []
all_auc = []

filename = 'email'

for i in range(10):
    ori_df = pd.read_csv('data/'+filename+'.txt', header=None, sep=' ')
    ori_df.columns = ['id_1', 'id_2']

    ori_G = nx.from_pandas_edgelist(ori_df, 'id_1', 'id_2')
    #print(nx.info(ori_G))
    
    negative_edges = get_negative_edges(ori_df, ori_G)
    #print(len(negative_edges))

    train_df, test_df = train_test_split(ori_df, test_size=0.1)
    
    nodes_number = len(ori_G.nodes)
    graph_np = np.zeros((nodes_number, nodes_number))
    for i in range(train_df.shape[0]):
        graph_np[train_df.iloc[i, 0], train_df.iloc[i, 1]] = 1
        graph_np[train_df.iloc[i, 1], train_df.iloc[i, 0]] = 1

    train_G = nx.from_numpy_matrix(graph_np)
    #print(nx.info(train_G))
    
    df_neg = pd.DataFrame(list(negative_edges), columns=['id_1', 'id_2'])
    
    train_df = train_df.values
    test_pos_df = test_df.values
    df_neg = df_neg.values
    
    model = LINE(train_G, embedding_size=12, order='all')
    model.train(batch_size=128, epochs=50, verbose=2)
    
    LINE_embeddings = model.get_embeddings()
    
    train_df = combine_embedding(train_df, LINE_embeddings)
    test_pos_df = combine_embedding(test_pos_df, LINE_embeddings)
    df_neg = combine_embedding(df_neg, LINE_embeddings)
    
    X_train_neg, X_test_neg = train_test_split(df_neg, test_size=0.1)
    
    y_train_pos = np.ones(len(train_df))
    y_train_neg = np.zeros(len(X_train_neg))
    y_test_pos = np.ones(len(test_pos_df))
    y_test_neg = np.zeros(len(X_test_neg))
    
    X_train = np.concatenate((train_df, X_train_neg))
    y_train = np.concatenate((y_train_pos, y_train_neg))
    X_test = np.concatenate((test_pos_df, X_test_neg))
    y_test = np.concatenate((y_test_pos, y_test_neg))
    
    clf = RandomForestClassifier(n_estimators=400)
    clf.fit(X_train, y_train)
    
    predict_Y = clf.predict(X_test)
    
    ap = average_precision_score(y_test, predict_Y)
    print("AP： ", ap)
    fpr, tpr, _ = metrics.roc_curve(y_test, predict_Y, pos_label=1)
    auc = metrics.auc(fpr, tpr)
    print("AUC SCORE: ",auc)
    
    all_ap.append(ap)
    all_auc.append(auc)

print("####################### SUM UP ############################")
print("AP MEAN : ", np.array(all_ap).mean())
print("AP STD : ", np.array(all_ap).std())
print("AUC MEAN : ", np.array(all_auc).mean())
print("AUC STD : ", np.array(all_auc).std())

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Epoch 1/50
678/678 - 1s - loss: 1.3422 - first_order_loss: 0.6931 - second_order_loss: 0.6491
Epoch 2/50
678/678 - 1s - loss: 1.1228 - first_order_loss: 0.6925 - second_order_loss: 0.4302
Epoch 3/50
678/678 - 1s - loss: 1.0657 - first_order_loss: 0.6904 - second_order_loss: 0.3753
Epoch 4/50
678/678 - 1s - loss: 1.0480 - first_order_loss: 0.6863 - second_order_loss: 0.3617
Epoch 5/50
678/678 - 1s - loss: 1.0376 - first_order_loss: 0.6819 - second_order_loss: 0.3557
Epoch 6/50
678/678 - 1s - loss: 1.0296 - first_order_loss: 0.6784 - second_order_loss: 0.3512
Epoch 7/50
678/678 - 1s - loss: 1.0247 - first_order_loss: 0.6751 - second_order_loss: 0.3497
Epoch 8/50
678/678 - 1s - loss: 1.0198 - first_order_loss: 0.6726 - se

Epoch 26/50
678/678 - 1s - loss: 0.9253 - first_order_loss: 0.6603 - second_order_loss: 0.2650
Epoch 27/50
678/678 - 1s - loss: 0.9223 - first_order_loss: 0.6609 - second_order_loss: 0.2613
Epoch 28/50
678/678 - 1s - loss: 0.9179 - first_order_loss: 0.6596 - second_order_loss: 0.2583
Epoch 29/50
678/678 - 1s - loss: 0.9147 - first_order_loss: 0.6592 - second_order_loss: 0.2555
Epoch 30/50
678/678 - 1s - loss: 0.9123 - first_order_loss: 0.6590 - second_order_loss: 0.2532
Epoch 31/50
678/678 - 1s - loss: 0.9103 - first_order_loss: 0.6595 - second_order_loss: 0.2508
Epoch 32/50
678/678 - 1s - loss: 0.9071 - first_order_loss: 0.6584 - second_order_loss: 0.2486
Epoch 33/50
678/678 - 1s - loss: 0.9036 - first_order_loss: 0.6586 - second_order_loss: 0.2450
Epoch 34/50
678/678 - 1s - loss: 0.9034 - first_order_loss: 0.6585 - second_order_loss: 0.2449
Epoch 35/50
678/678 - 1s - loss: 0.8990 - first_order_loss: 0.6578 - second_order_loss: 0.2412
Epoch 36/50
678/678 - 1s - loss: 0.8964 - first_or

Epoch 12/50
678/678 - 1s - loss: 1.0102 - first_order_loss: 0.6684 - second_order_loss: 0.3418
Epoch 13/50
678/678 - 1s - loss: 1.0060 - first_order_loss: 0.6675 - second_order_loss: 0.3385
Epoch 14/50
678/678 - 1s - loss: 1.0017 - first_order_loss: 0.6658 - second_order_loss: 0.3359
Epoch 15/50
678/678 - 1s - loss: 0.9972 - first_order_loss: 0.6648 - second_order_loss: 0.3324
Epoch 16/50
678/678 - 1s - loss: 0.9930 - first_order_loss: 0.6642 - second_order_loss: 0.3288
Epoch 17/50
678/678 - 1s - loss: 0.9877 - first_order_loss: 0.6646 - second_order_loss: 0.3231
Epoch 18/50
678/678 - 1s - loss: 0.9811 - first_order_loss: 0.6635 - second_order_loss: 0.3176
Epoch 19/50
678/678 - 1s - loss: 0.9729 - first_order_loss: 0.6619 - second_order_loss: 0.3111
Epoch 20/50
678/678 - 1s - loss: 0.9664 - first_order_loss: 0.6625 - second_order_loss: 0.3039
Epoch 21/50
678/678 - 1s - loss: 0.9595 - first_order_loss: 0.6621 - second_order_loss: 0.2973
Epoch 22/50
678/678 - 1s - loss: 0.9528 - first_or

Epoch 48/50
678/678 - 1s - loss: 0.8773 - first_order_loss: 0.6575 - second_order_loss: 0.2198
Epoch 49/50
678/678 - 1s - loss: 0.8764 - first_order_loss: 0.6571 - second_order_loss: 0.2193
Epoch 50/50
678/678 - 1s - loss: 0.8756 - first_order_loss: 0.6577 - second_order_loss: 0.2179
AP：  0.8022756453175586
AUC SCORE:  0.8522090852520225
Epoch 1/50
678/678 - 2s - loss: 1.3468 - first_order_loss: 0.6931 - second_order_loss: 0.6537
Epoch 2/50
678/678 - 1s - loss: 1.1279 - first_order_loss: 0.6926 - second_order_loss: 0.4353
Epoch 3/50
678/678 - 1s - loss: 1.0668 - first_order_loss: 0.6905 - second_order_loss: 0.3762
Epoch 4/50
678/678 - 1s - loss: 1.0498 - first_order_loss: 0.6871 - second_order_loss: 0.3627
Epoch 5/50
678/678 - 1s - loss: 1.0382 - first_order_loss: 0.6826 - second_order_loss: 0.3556
Epoch 6/50
678/678 - 1s - loss: 1.0315 - first_order_loss: 0.6789 - second_order_loss: 0.3526
Epoch 7/50
678/678 - 1s - loss: 1.0260 - first_order_loss: 0.6757 - second_order_loss: 0.3503
Ep

Epoch 34/50
678/678 - 1s - loss: 0.9070 - first_order_loss: 0.6589 - second_order_loss: 0.2480
Epoch 35/50
678/678 - 1s - loss: 0.9034 - first_order_loss: 0.6587 - second_order_loss: 0.2447
Epoch 36/50
678/678 - 1s - loss: 0.9017 - first_order_loss: 0.6585 - second_order_loss: 0.2432
Epoch 37/50
678/678 - 1s - loss: 0.9000 - first_order_loss: 0.6581 - second_order_loss: 0.2419
Epoch 38/50
678/678 - 1s - loss: 0.8979 - first_order_loss: 0.6585 - second_order_loss: 0.2395
Epoch 39/50
678/678 - 1s - loss: 0.8955 - first_order_loss: 0.6579 - second_order_loss: 0.2377
Epoch 40/50
678/678 - 1s - loss: 0.8956 - first_order_loss: 0.6586 - second_order_loss: 0.2370
Epoch 41/50
678/678 - 1s - loss: 0.8928 - first_order_loss: 0.6584 - second_order_loss: 0.2345
Epoch 42/50
678/678 - 1s - loss: 0.8913 - first_order_loss: 0.6584 - second_order_loss: 0.2329
Epoch 43/50
678/678 - 1s - loss: 0.8909 - first_order_loss: 0.6587 - second_order_loss: 0.2322
Epoch 44/50
678/678 - 1s - loss: 0.8874 - first_or

Epoch 20/50
678/678 - 1s - loss: 0.9695 - first_order_loss: 0.6622 - second_order_loss: 0.3073
Epoch 21/50
678/678 - 1s - loss: 0.9625 - first_order_loss: 0.6619 - second_order_loss: 0.3005
Epoch 22/50
678/678 - 1s - loss: 0.9568 - first_order_loss: 0.6616 - second_order_loss: 0.2952
Epoch 23/50
678/678 - 1s - loss: 0.9501 - first_order_loss: 0.6616 - second_order_loss: 0.2885
Epoch 24/50
678/678 - 1s - loss: 0.9437 - first_order_loss: 0.6607 - second_order_loss: 0.2830
Epoch 25/50
678/678 - 1s - loss: 0.9377 - first_order_loss: 0.6600 - second_order_loss: 0.2778
Epoch 26/50
678/678 - 2s - loss: 0.9331 - first_order_loss: 0.6601 - second_order_loss: 0.2730
Epoch 27/50
678/678 - 2s - loss: 0.9286 - first_order_loss: 0.6599 - second_order_loss: 0.2687
Epoch 28/50
678/678 - 1s - loss: 0.9244 - first_order_loss: 0.6601 - second_order_loss: 0.2643
Epoch 29/50
678/678 - 2s - loss: 0.9215 - first_order_loss: 0.6603 - second_order_loss: 0.2612
Epoch 30/50
678/678 - 2s - loss: 0.9152 - first_or